# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.dungeon.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState

player = DungeonPlayerState(
    playerId="BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,        
    weapon=MockFactory.create_weapon(),
    sub_weapon=MockFactory.create_sub_weapon(),
    inventory=[0, 21,100],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="스킬 사용법이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[1001],
}
# await graph.ainvoke(inputs, config=config)
# result = await graph.ainvoke({
#     "messages":[add_human_message("저 보스몬스터는 뭐야?")]
# }, config=config)

result = await graph.ainvoke({
    "messages":[add_human_message("무기 스왑 해줘")],
    "dungenon_player": player
}, config=config)

# result = await graph.ainvoke({
#     "messages":[add_human_message("인벤토리의 악세사리 뭐 있어?")],
#     "dungenon_player": player,
# }, config=config)

result["messages"]

총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "sub_weapon": {
    "weaponId": 20,
    "weaponType": 2,
    "weaponName": "일반 쌍검",
    "rarity": 0,
    "attackPower": 6,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>
    <INTERACTION_HANDLER>
        <INVENTORY_ITEMS>

Below is the data specification describing Items and Weapons
that the player can own, obtain, and equip.

Al

[HumanMessage(content='내 공격력이 몇이야?', additional_kwargs={'created_at': '2025-12-19T09:40:34.396405'}, response_metadata={}, id='8be84308-f3fc-470e-aedc-d3a45614eea7'),
 AIMessage(content='지금 장착된 레어 쌍검의 공격력은 11이야!', additional_kwargs={'created_at': '2025-12-19T09:40:35.897219', 'intent_types': ['INTERACTION_HANDLER']}, response_metadata={}, id='b405b61c-21eb-43a3-990b-a457e6835693')]

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [ ]:
result = await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?"),
     "dungenon_player": player,
     "target_monster_ids":[0,1],
     }, config=config
)   

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="다음방에 뭐있는지 확인해봐"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [ ]:
player = DungeonPlayerState(
    playerId="BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
    heroineId=1,
    currRoomId=3,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },

    config=config,
)
result["messages"]

# 전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
# expected_outcome='1번 선택은 기억의 고통을 정면으로 마주해 정신적 타격(instant_damage_medium)을 입지만, 트라우마를 전투 감각으로 전환해 스킬 데미지(skill_damage_increase_all)가 크게 상승한다.\n2번 선택은 죄책감을 회피해 즉각적인 피해는 피하지만, 마음이 무거운 족쇄가 되어 이동속도(move_speed_decrease_all)가 감소해 전투와 탐사에서 기동력이 떨어진다.\n3번 선택은 분노로 정신을 끊어내며 일시적 피해(instant_damage_low)를 받는 대신, 신경이 과도하게 흥분해 공격 속도(attack_speed_increase_all)가 빨라져 근접 교전에 유리해진다.\n4번 선택은 죄책감을 인정하고 상처를 들여다보며 내면이 단단해져 최대 체력(hp_increase_all)이 상승하지만, 기억의 잔향이 스킬 운용을 방해해 일정 시간 스킬 데미지 감소(curse_debuff)를 겪게 된다.') floor=1



## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [ ]:
player = DungeonPlayerState(
    playerId="BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
    heroineId=1,
    currRoomId=2,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이 던전에 몬스터 뭐 나와?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]


In [ ]:
result = await graph.ainvoke(
    {
        "messages": add_human_message(content="저거 패턴좀 알려줘"),
        "dungenon_player": player,
        'target_monster_ids':[1001]
    },
    config=config,
)
result["messages"]

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?"),
     "dungenon_player": player, },
    config=config
)
result["messages"]

In [ ]:
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="그럼 지금 보스방은 못가?"),
#      "dungenon_player": player, },
#     config=config
# )
result = await graph.ainvoke(
    { "messages": add_human_message(content="보스방울 어떻게 봐?"),
     "dungenon_player": player, },
    config=config
)


In [ ]:
result["messages"][-1]

In [ ]:
from agents.fairy.interaction.fairy_interaction_agent import graph_builder as interaction_builder
interaction_graph = interaction_builder.compile() 
interaction_graph.invoke({
    "messages": add_human_message(content="다음방 확인해"),
    "inventory": [0, 21, 42],
})

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.guild.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
guild_graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
result = await guild_graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는 힌트좀 줘") },
    config=config
)
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
#     config=config
# )
result["messages"]

In [ ]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="왜 던전에서 사냥해야해? 이유가 있어?") },
    config=config
)

In [ ]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="나 이제 뭐 해야돼?") },
    config=config
)

In [ ]:
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 100,
                    " affection":100, "sanity": 100, "thread_id": "100"}
}
await guild_graph.ainvoke(
    { "messages": add_human_message(content="모든 메모리를 해제했엉 이제 나 뭐해야해?") },
    config=config
)